<a href="https://colab.research.google.com/github/gamecicn/TF2_Study/blob/KaggleMinst/T2E1_Kaggle_MINST_CNNv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd 
import numpy as np
import tensorflow as tf

In [4]:
# Download Data
csv_train_file = '/content/drive/MyDrive/DS_data/MINSTHW/train.csv'
csv_test_file = '/content/drive/MyDrive/DS_data/MINSTHW/test.csv'

df_data = pd.read_csv(csv_train_file)
df_test = pd.read_csv(csv_test_file)

df_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Preprcessing

In [5]:
# Split train data to train and valid dataset
df_data_copy = df_data.copy()

df_train  = df_data_copy.sample(frac=0.99, random_state=0)
df_valid = df_data_copy.drop(df_train.index)

print("lenght of train is {}".format(len(df_train)))
print("lenght of valid is {}".format(len(df_valid)))

train_labels = df_train.pop('label')
valid_labels = df_valid.pop('label')

df_train.head()

#np.apply_along_axis(lambda x : np.reshape, 1, df_train.to_numpy(), 28, 28)
df_train = df_train.to_numpy().reshape(-1, 28 ,28, 1)
df_valid = df_valid.to_numpy().reshape(-1, 28 ,28, 1)

lenght of train is 41580
lenght of valid is 420


## Build the model

In [8]:
'''
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(28, (2, 2), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(48, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(48, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()
'''

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(28, (5, 5), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(48, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 28)        728       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 28)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 28)        112       
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 48)          33648     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 48)          0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 4, 4, 48)         

### Training

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(df_train, 
                    train_labels,
                    batch_size=64,
                    epochs=50)

Epoch 1/50
650/650 [==============================] - 9s 3ms/step - loss: 0.3147 - accuracy: 0.9007
Epoch 2/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0547 - accuracy: 0.9821
Epoch 3/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0388 - accuracy: 0.9875
Epoch 4/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0301 - accuracy: 0.9902
Epoch 5/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0220 - accuracy: 0.9924
Epoch 6/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0188 - accuracy: 0.9940
Epoch 7/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0222 - accuracy: 0.9925
Epoch 8/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0168 - accuracy: 0.9937
Epoch 9/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0138 - accuracy: 0.9955
Epoch 10/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0139 - accuracy: 0.9952

### Validate

In [10]:
 val_loss, val_acc = model.evaluate(df_valid,  valid_labels, verbose=2)

14/14 - 0s - loss: 0.0860 - accuracy: 0.9881


## Make predictions

In [11]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(df_test.to_numpy().reshape(-1, 28, 28, 1))

res = pd.DataFrame(data = {"ImageId" : np.arange(1, 1+len(predictions)) , 
                           "Label"   : list(map(np.argmax, predictions))})

In [12]:


res.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [14]:
res.to_csv("/content/drive/MyDrive/output/Kaggle_minst_hw_cnn5.csv", index=False)